In [13]:
import pandas as pd
#adding first line

# Class FB Member

In [1]:
import time
from bs4 import BeautifulSoup

class FBMember(object):

    def __init__(self, driver):
        self.__driver = driver

    def login(self, user_name, password):
        self.__driver.get('https://web.facebook.com/')

        username_box = self.__driver.find_element_by_id('email')
        username_box.send_keys(user_name)
        print("Email Id entered")
        time.sleep(1)

        password_box = self.__driver.find_element_by_id('pass')
        password_box.send_keys(password)
        print("Password entered")
        time.sleep(1)

        login_box = self.__driver.find_element_by_id('loginbutton')
        login_box.click()

        time.sleep(20)

    def scroll(self, pause_time):
        # Get scroll height
        last_height = self.__driver.execute_script('return document.body.scrollHeight')
        while True:
            # scroll to bottom
            self.__driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # wait loading page
            time.sleep(pause_time)

            new_height = self.__driver.execute_script('return document.body.scrollHeight')
            if new_height == last_height:
                break
            last_height = new_height

    def get_group_members(self, group_id):
        self.__driver.get('https://web.facebook.com/groups/' + group_id + '/members')

        # for k in range(1, 5):
        #     self.__driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        #     time.sleep(3)

        self.scroll(3)

        html = self.__driver.execute_script('return document.documentElement.outerHTML')
        fb_html = BeautifulSoup(html, 'html.parser')
        usr = fb_html.findAll('div', {'class': "_60ri"})
        usr_url = [a.find('a').get('href') for a in usr]
        return usr_url

# Class FB Post

In [27]:
import datetime as dt
import time
import ast
from bs4 import BeautifulSoup

class FBPosts(object):

    def __init__(self, driver, days_lim):
        self.__driver = driver
        self.__days_lim = days_lim

    def __get_date(self, post):
        str_post_timestamp = post.get('data-store')
        # convert timestamp to dictionary
        post_timestamp = ast.literal_eval(str_post_timestamp)
        post_date = dt.date.fromtimestamp(post_timestamp.get('timestamp'))
        return post_date

    def get_user_post(self, user_url):

        days_lim = self.__days_lim
        date_today = dt.date.today()

        self.__driver.get(user_url)
        time.sleep(5)

        days_diff = 0
        contents = []

        # cek apakah post terakhir melebihi days_lim
        src1 = self.__driver.page_source
        soup1 = BeautifulSoup(src1, 'html.parser')
        post1 = soup1.find('div', attrs={'_5pcb _4b0l _2q8l'})
        
        post_by = soup1.find('a', attrs={'_2nlw _2nlv'}).get_text()
        
        if post1:
            first_date = self.__get_date(post1)
            days_diff1 = (date_today - first_date).days
            if days_diff1 > days_lim:
                pass
            else:
                # Looping untuk scrolling
                while days_diff < days_lim:
                    # giving 5 seconds to load the page
                    time.sleep(3)
                    # scroll to the bottom of the page
                    self.__driver.execute_script(
                        "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
                    # getting the page source
                    src = self.__driver.page_source
                    soup = BeautifulSoup(src, 'html.parser')
                    posts = soup.find_all('div', attrs={'_5pcb _4b0l _2q8l'})
                    # getting timestamp (str)
                    str_earliest_timestamp = posts[-1].get('data-store')
                    # convert timestamp to dictionary
                    earliest_timestamp = ast.literal_eval(str_earliest_timestamp)
                    earliest_date = dt.date.fromtimestamp(earliest_timestamp.get('timestamp'))
                    days_diff = (date_today - earliest_date).days

                # menghapus post yang lebih lama dari days_lim
                earliest_date = self.__get_date(posts[-1])
                days_diff2 = (date_today - earliest_date).days
                while days_diff2 > days_lim:
                    posts.remove(posts[-1])
                    earliest_date = self.__get_date(posts[-1])
                    days_diff2 = (date_today - earliest_date).days

                for post in posts:
                    # deleting element 'Lihat delengkapnya' & 'Lihat terjemahan'
                    for span in post.find_all('span', attrs={'text_exposed_hide'}):
                        span.decompose()
                    for div in post.find_all('div', {'class': '_43f9 _63qh'}):
                        div.decompose()
                    post_date = self.__get_date(post)
                    
                    #extract content
                    content_soup = post.find('div', {'data-testid':'post_message'})
                    if content_soup:
                        content = content_soup.get_text()
                    else:
                        continue
                    
                    #extract jumlah likes dan comments
                    likes_tag = post.find('span',attrs={'_81hb'})
                    if likes_tag:
                        num_likes = likes_tag.get_text()
                    else:
                        num_likes = 0
                    
                    comments_tag = post.find('a',attrs={'_3hg- _42ft'})
                    if comments_tag:
                        num_comments = comments_tag.get_text().split(' ')[0]
                    else:
                        num_comments = 0
                    
                    content_info = {
                        'post_date':post_date,
                        'post_by': post_by,
                        'content': content,
                        'likes':num_likes,
                        'comments':num_comments
                    }
                    contents.append(content_info)
        return contents

# Aplikasi

## Variabel

In [3]:
from selenium import webdriver

driver = webdriver.Chrome(executable_path = "C:/Program Files (x86)/chromedriver.exe")

username = "sukontoraharjo@gmail.com"
password = "akun0001"
group_id = "743158729087573"

fb_member = FBMember(driver = driver)

fb_post = FBPosts(driver=driver, days_lim=20)

group_list = [group_id]

## Log in

In [4]:
fb_member.login(username, password)

Email Id entered
Password entered


## Get Member

In [7]:
for group in group_list:
    user_list = fb_member.get_group_members(group)
print(len(user_list))

1968


In [9]:
user_list[10]

'https://www.facebook.com/profile.php?id=100005244406927&fref=gm&__tn__=%2Cd-_-R&eid=ARAUf04RoCvmiEd2_Gqxbt8x-vk3vP06Rtqzhj3N-GhvU3BF4p2K3CM2-Z3Y9xAb7JUB_X7vzoiQb-iI&dti=743158729087573&hc_location=group'

## Get Post

In [28]:
fb_post = FBPosts(driver=driver, days_lim=10)
all_user_post = []
for user in [user_list[10]]:
    all_user_post = all_user_post+fb_post.get_user_post(user)

In [29]:
pd.DataFrame(all_user_post)

,post_date,post_by,content,likes,comments
0,2020-07-23,Rrusman (Chie baudt thea),Jadi amek kapegung kie aing..,5,6
1,2020-07-23,Rrusman (Chie baudt thea),Bgg,3,0
2,2020-07-21,Rrusman (Chie baudt thea),ker jalangarteh aya we jelema nu pibangusen😠,13,0
3,2020-07-21,Rrusman (Chie baudt thea),Bilih aya numndakan kunci motor aya tali cokla...,32,8
4,2020-07-21,Rrusman (Chie baudt thea),cuacana mantep,21,2
5,2020-07-20,Rrusman (Chie baudt thea),"usum kiemh nu penting ngopi,dahar,budak weh ka...",29,15
6,2020-07-19,Rrusman (Chie baudt thea),Lumanglimung sorangan.tepuguh eweh budak mch..,32,4
7,2020-07-14,Rrusman (Chie baudt thea),Kudu jadi contoh nu sk nyimpen hp dina jok motor🙏,0,0
8,2020-07-14,Rrusman (Chie baudt thea),Kareasel,11,0
9,2020-07-13,Rrusman (Chie baudt thea),"BUTUH LOKER,,",17,0


In [16]:
src = driver.page_source
soup = BeautifulSoup(src, 'html.parser')
posts = soup.find_all('div', attrs={'_5pcb _4b0l _2q8l'})

In [22]:
posts[0].find('span',attrs={'_81hb'}).get_text()

'5'